# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


In [ ]:
%help

####  Run this cell to set up and start your interactive session.


In [1]:
%idle_timeout 2880
%glue_version 5.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.8 
Current idle_timeout is None minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 5.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 5
Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 5
Idle Timeout: 2880
Session ID: 3ef00648-2f08-42f5-96b6-155dbdeaa765
Applying the following default arguments:
--glue_kernel_version 1.0.8
--enable-glue-datacatalog true
Waiting for session 3ef00648-2f08-42f5-96b6-155dbdeaa765 to get into ready status...
Session 3ef00648-2f08-42f5-96b6-155dbdeaa765 ha

#### Example: Create a DynamicFrame from a table in the AWS Glue Data Catalog and display its schema


In [2]:
from datetime import datetime
from pyspark.sql import SparkSession, functions as F

spark = SparkSession.builder.appName("bronze-date-dim-fix").getOrCreate()

# Avoid $folder$ marker writes if your IAM is strict
spark._jsc.hadoopConfiguration().set("fs.s3a.create.directory", "false")

BUCKET = "bus-insights-dev-us-east-1"

# We want both partitions to mean "business date", i.e., ingestion_date == date_key
FIX_DATES = ["2025-09-27", "2024-02-22"]  # exactly your two partitions

def rewrite_date_dim(day: str):
    dt = datetime.strptime(day, "%Y-%m-%d")
    df = spark.createDataFrame([{
        "date_key": day,
        "year": dt.year,
        "month": dt.month,
        "day": dt.day,
        "week": int(dt.strftime("%V")),       # ISO week
        "day_of_week": dt.strftime("%A"),
        "is_weekend": dt.weekday() >= 5,
        "is_holiday": False,
        "holiday_name": None
    }])
    (df.withColumn("ingestion_date", F.lit(day))
       .write.mode("overwrite")
       .option("compression","snappy")
       .parquet(f"s3://{BUCKET}/bronze/date_dim/ingestion_date={day}/"))
    print(f"Rewrote bronze/date_dim for {day}")

for d in FIX_DATES:
    rewrite_date_dim(d)


PySparkValueError: [CANNOT_DETERMINE_TYPE] Some of types cannot be determined after inferring.


In [3]:
from datetime import datetime
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import (
    StructType, StructField, StringType, IntegerType, BooleanType
)

spark = SparkSession.builder.appName("bronze-date-dim-fix").getOrCreate()
# Optional: avoid $folder$ markers if your IAM is strict
spark._jsc.hadoopConfiguration().set("fs.s3a.create.directory", "false")

BUCKET = "bus-insights-dev-us-east-1"
FIX_DATES = ["2025-09-27", "2024-02-22"]  # your two partitions

# Explicit schema (matches what we want in Bronze)
schema = StructType([
    StructField("date_key",     StringType(),  False),
    StructField("year",         IntegerType(), False),
    StructField("month",        IntegerType(), False),
    StructField("day",          IntegerType(), False),
    StructField("week",         IntegerType(), False),
    StructField("day_of_week",  StringType(),  False),
    StructField("is_weekend",   BooleanType(), False),
    StructField("is_holiday",   BooleanType(), False),
    StructField("holiday_name", StringType(),  True),
    # keep ingestion_date in-file as STRING to match your Bronze table metadata
    StructField("ingestion_date", StringType(), False),
])

def rewrite_date_dim(day: str):
    dt = datetime.strptime(day, "%Y-%m-%d")
    rows = [{
        "date_key": day,
        "year": dt.year,
        "month": dt.month,
        "day": dt.day,
        "week": int(dt.strftime("%V")),        # ISO week number
        "day_of_week": dt.strftime("%A"),
        "is_weekend": dt.weekday() >= 5,
        "is_holiday": False,
        "holiday_name": None,
        "ingestion_date": day                  # keep semantics aligned
    }]
    df = spark.createDataFrame(rows, schema=schema)

    out = f"s3://{BUCKET}/bronze/date_dim/ingestion_date={day}/"
    (df.write.mode("overwrite").option("compression","snappy").parquet(out))
    print(f"Rewrote bronze/date_dim for {day} -> {out}")

for d in FIX_DATES:
    rewrite_date_dim(d)


Rewrote bronze/date_dim for 2025-09-27 -> s3://bus-insights-dev-us-east-1/bronze/date_dim/ingestion_date=2025-09-27/
Rewrote bronze/date_dim for 2024-02-22 -> s3://bus-insights-dev-us-east-1/bronze/date_dim/ingestion_date=2024-02-22/


In [4]:
# Read both partitions back and show schema/preview
base = f"s3://{BUCKET}/bronze/date_dim/"
df_all = spark.read.option("basePath", base).parquet(base)
df_all.printSchema()
df_all.orderBy("ingestion_date").show(5, False)


root
 |-- date_key: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- is_weekend: boolean (nullable = true)
 |-- is_holiday: boolean (nullable = true)
 |-- holiday_name: string (nullable = true)
 |-- ingestion_date: date (nullable = true)

+----------+----+-----+---+----+-----------+----------+----------+------------+--------------+
|date_key  |year|month|day|week|day_of_week|is_weekend|is_holiday|holiday_name|ingestion_date|
+----------+----+-----+---+----+-----------+----------+----------+------------+--------------+
|2024-02-22|2024|2    |22 |8   |Thursday   |false     |false     |NULL        |2024-02-22    |
|2025-09-27|2025|9    |27 |39  |Saturday   |true      |false     |NULL        |2025-09-27    |
+----------+----+-----+---+----+-----------+----------+----------+------------+--------------+
